<a href="https://colab.research.google.com/github/Monika171/Deep-Learning-with-Pytorch/blob/main/logisticregression_softmax_cross_entropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.utils.colab.set_colab_file_id('1BEmaZMjxkgi8bmpfYn1gi76_-YcVrc68')

     |████████████████████████████████| 71kB 2.9MB/s 


In [2]:
!pip install jovian --upgrade --quiet

In [3]:
jovian.commit(project='logistic-regression-softmax-crossentropy')

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ··········
[jovian] Uploading colab notebook to Jovian...
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/monika171/logistic-regression-softmax-crossentropy


'https://jovian.ai/monika171/logistic-regression-softmax-crossentropy'